In [1]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox

In [2]:
import cv2
import numpy as np
from PIL import Image
import os

In [ ]:

NAMES_FILE = "names.txt"  # File to store ID and name pairs

def generate_dataset(name, user_id):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None
        
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        ret, frame = cap.read()
        cropped = face_cropped(frame)
        
        if cropped is not None:
            img_id += 1
            face = cv2.resize(cropped, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            file_path = f"data/user.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or img_id == 100:
                break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Dataset Creation Completed", f"Dataset creation for {name} (ID: {user_id}) completed.")


def train_classifier(data_dir="data"):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training completed.")

def draw_boundary(image, classifier, scaleFactor, minNeighbors, color, clf):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    coords = []
    
    with open(NAMES_FILE, "r") as f:
        name_dict = {int(line.split(",")[0]): line.split(",")[1].strip() for line in f.readlines()}
    
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_image[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 72:
            name = name_dict.get(id, "Unknown")
            cv2.putText(image, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        else:
            cv2.putText(image, "unknown", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), clf)
    return img

def start_recognition(camera_choice):
    cam_index = 0 if camera_choice == "Webcam" else 1

    try:
        video_capture = cv2.VideoCapture(cam_index)
        if not video_capture.isOpened():
            raise Exception(f"Camera with index {cam_index} could not be opened.")
        
        faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        clf = cv2.face.LBPHFaceRecognizer_create()
        clf.read("classifier.xml")

        while True:
            ret, img = video_capture.read()
            if not ret:
                raise Exception("Failed to read from the camera. Please check the connection.")
            
            img = recognize(img, clf, faceCascade)
            cv2.imshow("Face Detection", img)

            if cv2.waitKey(1) == 13:
                break

    except Exception as e:
        messagebox.showerror("Recognition Error", str(e))

    finally:
        if 'video_capture' in locals() and video_capture.isOpened():
            video_capture.release()
        cv2.destroyAllWindows()

def add_new_face():
    name = simpledialog.askstring("Input", "Enter your name:")
    if name:
        # Assign a new ID based on the current dataset
        existing_files = os.listdir("data")
        user_id = len(existing_files) // 100 + 1  # Assuming 100 images per user

        # Save the name with the associated ID
        with open(NAMES_FILE, "a") as f:
            f.write(f"{user_id},{name}\n")

        generate_dataset(name, user_id)
        train_classifier()

def main():
    root = tk.Tk()
    root.geometry("600x400")
    root.title("Face Recognizer")
    root.configure(bg="gray")
    root.resizable(False, False)

    label = tk.Label(root, text="WELCOME TO FACE RECOGNIZER...", font=('Times New Roman', 20, 'bold'), bg="gray")
    label.pack(padx=40, pady=20)

    camera_label = tk.Label(root, text="Select The Camera:", font=('Times New Roman', 15, 'bold'), bg="gray")
    camera_label.pack(pady=10)

    camera_options = ["Webcam", "External Cam"]
    camera_select = ttk.Combobox(root, values=camera_options, font=('Georgia', 14), state="readonly")
    camera_select.current(0)
    camera_select.pack(pady=20)

    login_button = tk.Button(root, text="Detect Face", command=lambda: start_recognition(camera_select.get()), font=('Georgia', 14, 'bold'), bg="white")
    login_button.pack(pady=10)

    label = tk.Label(root, text="or", font=('Times New Roman', 15, 'bold'), bg="gray")
    label.pack(padx=5, pady=5)

    create_button = tk.Button(root, text="Add Face", command=add_new_face, font=('Georgia', 14, 'bold'), bg="white")
    create_button.pack(pady=10)

    root.mainloop()

main()

Training completed.
